# MORDM

Many Objectives Decision Making

MORDM has four main steps:

(i) problem formulation based on a systems analytical problem definition framework

(ii) searching for candidate solutions that optimize multiple objectives by using multi-objective evolutionary algorithms

(iii) generating an ensemble of scenarios to explore the effects of uncertainties

(iv) using scenario discovery to detect the vulnerabilities of candidate solutions and improving thecandidate solutions

In [15]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    perform_experiments,
    Samplers,
    Scenario
)
import numpy as np
import pandas as pd
import copy
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time, min_over, guaranteed95_over, guaranteed98_over

from ema_workbench.em_framework.optimization import EpsilonProgress


In [16]:
# Select problem formulation 3 - Water Levels
ema_logging.log_to_stderr(ema_logging.INFO)
dike_model, planning_steps = get_model_for_problem_formulation(3)

## Problem Formulation


The room for the river model has 6 outcome indicators. The first four indicators are specific to each dike and the last two concern the model as a whole:
- Expected Annual Damage
- Expected Number of Deaths
- Dike Investment Costs
- Water Level
- RfR Total Costs
- Expected Evacuation Costs

As analysts for the transport company, the capacity to transport goods in the river is heavily dependent on the water level staying above a threshold. Therefore, it is crucial for the problem owner to focus on the policies where this target is met. Nonetheless, based on the insights provided by the different stakeholders during the debate, it becomes imperative to explore options that have the desired water level with the least amount of casualties and costs.

In [17]:
# Model Uncertainties
for uncertainty in dike_model.uncertainties:
    print(repr(uncertainty))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [18]:
# Policy Levers
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [19]:
# Model Outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x0000018DA135AFC0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x0000018DA135AFC0>)
ScalarOutcome('A.1_Minimum Water Level', variable_name=('A.1_Water Level',), function=<function min_over at 0x0000018DA1C36B60>)
ScalarOutcome('A.1_95% Guaranteed Water Level', variable_name=('A.1_Water Level',), function=<function guaranteed95_over at 0x0000018DA1C50EA0>)
ScalarOutcome('A.1_98% Guaranteed Water Level', variable_name=('A.1_Water Level',), function=<function guaranteed98_over at 0x0000018DA1C50F40>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x0000018DA135AFC0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<funct

## Candidate Strategies

### Search over levers
Direct search is used to go through the decision levers to find good candidate strategies that meet the desired outcomes of the problem owner.

In [20]:
# Define a reference scenario (This scenario came by default on the model)
reference_values = {
        "Bmax": 175,
        "Brate": 1.5,
        "pfail": 0.5,
        "discount rate 0": 3.5,
        "discount rate 1": 3.5,
        "discount rate 2": 3.5,
        "ID flood wave shape": 4,
    }
scen1 = {}
for key in dike_model.uncertainties:
    name_split = key.name.split("_")

    if len(name_split) == 1:
        scen1.update({key.name: reference_values[key.name]})

    else:
        scen1.update({key.name: reference_values[name_split[1]]})

ref_scenario = Scenario("reference", **scen1)

In [21]:
# Define metrics for the optimization
convergence_metrics = [EpsilonProgress()]
epsilon = [1e3] * len(dike_model.outcomes)
nfe = 200  # proof of principle only, way to low for actual use

with MultiprocessingEvaluator(dike_model) as evaluator:
    results, convergence = evaluator.optimize(
        nfe=nfe,
        searchover="levers",
        epsilons=epsilon,
        convergence=convergence_metrics,
        reference=ref_scenario,
        )

[MainProcess/INFO] pool started with 12 workers
100%|████████████████████████████████████████| 200/200 [00:16<00:00, 12.01it/s]
[MainProcess/INFO] optimization completed, found 97 solutions
[MainProcess/INFO] terminating pool


In [23]:
results
results.to_excel('MORDM.xlsx')

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)
# Evolutionary algorithm
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.optimize(nfe=250, searchover="levers", epsilons=[0.1] * len(dike_model.outcomes))

[MainProcess/INFO] pool started with 12 workers
  0%|                                                  | 0/250 [00:00<?, ?it/s]